# 🧬 Genomics Data Analysis Template

**Cloud-Native Multiomics Platform - Vertex AI Workbench**

This notebook provides a template for exploring genomic data stored in BigQuery and running custom analyses.

## Contents
1. [Setup & Authentication](#1-setup)
2. [Query Genomic Variants](#2-query-variants)
3. [Population Statistics](#3-population-stats)
4. [Visualization](#4-visualization)
5. [Export for ML Training](#5-ml-export)

## 1. Setup & Authentication <a name="1-setup"></a>

In [ ]:
# Install required packages (if not already installed)
# !pip install google-cloud-bigquery pandas pyarrow matplotlib seaborn plotly

In [ ]:
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Configuration
PROJECT_ID = os.environ.get('GOOGLE_CLOUD_PROJECT', 'YOUR_PROJECT_ID')
DATASET_ID = 'genomics_warehouse'
LOCATION = 'US'

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

print(f"Connected to project: {PROJECT_ID}")
print(f"Dataset: {DATASET_ID}")

## 2. Query Genomic Variants <a name="2-query-variants"></a>

In [ ]:
# Query BRCA1 variants from 1000 Genomes public data
BRCA1_QUERY = """
SELECT
    reference_name,
    start_position,
    end_position,
    reference_bases,
    alternate_bases,
    names AS variant_ids,
    quality
FROM
    `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`
WHERE
    reference_name = '17'
    AND start_position BETWEEN 41196312 AND 41277500
ORDER BY
    start_position
LIMIT 1000
"""

brca1_df = client.query(BRCA1_QUERY).to_dataframe()
print(f"Found {len(brca1_df)} BRCA1 variants")
brca1_df.head(10)

In [ ]:
# Helper function for running queries
def query_bq(sql: str, use_cache: bool = True) -> pd.DataFrame:
    """Execute a BigQuery query and return a DataFrame."""
    job_config = bigquery.QueryJobConfig(use_query_cache=use_cache)
    df = client.query(sql, job_config=job_config).to_dataframe()
    return df

## 3. Population Statistics <a name="3-population-stats"></a>

In [ ]:
# Variant counts by chromosome
CHROMOSOME_COUNTS_QUERY = """
SELECT
    reference_name,
    COUNT(*) AS variant_count
FROM
    `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`
WHERE
    reference_name IN ('1','2','3','4','5','6','7','8','9','10',
                       '11','12','13','14','15','16','17','18','19','20','21','22','X','Y')
GROUP BY
    reference_name
ORDER BY
    CASE reference_name
        WHEN 'X' THEN 23
        WHEN 'Y' THEN 24
        ELSE CAST(reference_name AS INT64)
    END
"""

chrom_counts = query_bq(CHROMOSOME_COUNTS_QUERY)
chrom_counts

In [ ]:
# Transition/Transversion ratio (Ti/Tv)
TITV_QUERY = """
SELECT
    CASE
        WHEN (reference_bases = 'A' AND alternate_bases[OFFSET(0)] = 'G') OR
             (reference_bases = 'G' AND alternate_bases[OFFSET(0)] = 'A') OR
             (reference_bases = 'C' AND alternate_bases[OFFSET(0)] = 'T') OR
             (reference_bases = 'T' AND alternate_bases[OFFSET(0)] = 'C')
        THEN 'Transition'
        ELSE 'Transversion'
    END AS mutation_type,
    COUNT(*) AS count
FROM
    `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`
WHERE
    LENGTH(reference_bases) = 1
    AND ARRAY_LENGTH(alternate_bases) = 1
    AND LENGTH(alternate_bases[OFFSET(0)]) = 1
GROUP BY
    mutation_type
"""

titv_df = query_bq(TITV_QUERY)
ti = titv_df[titv_df['mutation_type'] == 'Transition']['count'].values[0]
tv = titv_df[titv_df['mutation_type'] == 'Transversion']['count'].values[0]
print(f"Ti/Tv Ratio: {ti/tv:.3f}")
print(f"(Expected: ~2.0 for WGS, ~2.5 for WES)")
titv_df

## 4. Visualization <a name="4-visualization"></a>

In [ ]:
# Configure plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Variant distribution across chromosomes
fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(chrom_counts['reference_name'], chrom_counts['variant_count'] / 1e6)
ax.set_xlabel('Chromosome', fontsize=12)
ax.set_ylabel('Variant Count (millions)', fontsize=12)
ax.set_title('Variant Distribution Across Chromosomes (1000 Genomes Phase 3)', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# BRCA1 variant quality distribution
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(brca1_df['quality'].dropna(), bins=50, edgecolor='black', alpha=0.7)
ax.set_xlabel('QUAL Score', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('BRCA1 Variant Quality Score Distribution', fontsize=14)
ax.axvline(x=30, color='red', linestyle='--', label='QUAL=30 threshold')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Interactive plot with Plotly
fig = px.bar(
    chrom_counts,
    x='reference_name',
    y='variant_count',
    title='Interactive: Variant Counts by Chromosome',
    labels={'reference_name': 'Chromosome', 'variant_count': 'Variant Count'},
    color='variant_count',
    color_continuous_scale='Viridis'
)
fig.show()

## 5. Export for ML Training <a name="5-ml-export"></a>

In [ ]:
# Export query results to GCS for ML training
def export_to_gcs(
    query: str,
    destination_uri: str,
    export_format: str = 'CSV'
):
    """Export BigQuery query results to GCS."""
    
    # First, save query results to a temp table
    job_config = bigquery.QueryJobConfig(
        destination=f"{PROJECT_ID}.{DATASET_ID}.temp_export_{int(pd.Timestamp.now().timestamp())}",
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    )
    
    query_job = client.query(query, job_config=job_config)
    query_job.result()  # Wait for query to complete
    
    # Export to GCS
    destination_table = query_job.destination
    
    extract_config = bigquery.ExtractJobConfig(
        destination_format=getattr(bigquery.DestinationFormat, export_format),
        compression=bigquery.Compression.GZIP if export_format != 'AVRO' else bigquery.Compression.SNAPPY,
    )
    
    extract_job = client.extract_table(
        destination_table,
        destination_uri,
        job_config=extract_config,
    )
    extract_job.result()  # Wait for export
    
    # Clean up temp table
    client.delete_table(destination_table)
    
    print(f"✓ Exported to: {destination_uri}")

# Example: Export BRCA1 variants for ML
# export_to_gcs(
#     BRCA1_QUERY,
#     f"gs://{PROJECT_ID}-multiomics-results-dev/exports/brca1_variants_*.csv.gz"
# )

In [ ]:
# Create training dataset for variant classification
ML_FEATURES_QUERY = """
WITH variant_features AS (
    SELECT
        reference_name,
        start_position,
        reference_bases,
        alternate_bases[OFFSET(0)] AS alt,
        quality,
        -- Derived features
        LENGTH(reference_bases) AS ref_length,
        LENGTH(alternate_bases[OFFSET(0)]) AS alt_length,
        CASE
            WHEN LENGTH(reference_bases) = LENGTH(alternate_bases[OFFSET(0)]) THEN 'SNP'
            WHEN LENGTH(reference_bases) > LENGTH(alternate_bases[OFFSET(0)]) THEN 'DEL'
            ELSE 'INS'
        END AS variant_type,
        -- Label (for supervised learning, needs external annotation)
        -- You would join with ClinVar or similar for labels
    FROM
        `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`
    WHERE
        ARRAY_LENGTH(alternate_bases) = 1
        AND quality IS NOT NULL
    LIMIT 100000
)
SELECT * FROM variant_features
"""

ml_df = query_bq(ML_FEATURES_QUERY)
print(f"Training dataset shape: {ml_df.shape}")
ml_df.head()

In [ ]:
# Feature distribution for ML dataset
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Variant type distribution
ml_df['variant_type'].value_counts().plot(kind='bar', ax=axes[0])
axes[0].set_title('Variant Type Distribution')
axes[0].set_xlabel('Type')
axes[0].set_ylabel('Count')

# Quality score distribution
axes[1].hist(ml_df['quality'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Quality Score Distribution')
axes[1].set_xlabel('QUAL')
axes[1].set_ylabel('Frequency')

# Reference length distribution
axes[2].hist(ml_df['ref_length'], bins=20, edgecolor='black', alpha=0.7)
axes[2].set_title('Reference Allele Length')
axes[2].set_xlabel('Length (bp)')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

---

## Next Steps

1. **Join with annotations**: Combine with ClinVar or gnomAD for clinical significance labels
2. **Train ML model**: Use Vertex AI AutoML or custom TensorFlow/PyTorch models
3. **Deploy predictions**: Serve variant pathogenicity predictions via Vertex AI Endpoints

---

*Cloud-Native Multiomics Platform - Built with ❤️ on Google Cloud*